<a href="https://colab.research.google.com/github/jnels4/nfl_predictions/blob/john/NFL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-12 23:52:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-09-12 23:52:15 (10.8 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [29]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NFL-Prediction").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [31]:
from pyspark import SparkFiles
url = "https://nflpredictionsdataviz.s3.amazonaws.com/spreadspoke_scores.csv"
spark.sparkContext.addFile(url)
NFL_Guru_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("spreadspoke_scores.csv"), sep=",", header=True, inferSchema=True)
NFL_Guru_df.show()

+-------------+---------------+-------------+----------------+-------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|          team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+-------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|     9/2/1966|           1966|            1|           false|     Miami Dolphins|        14|        23|     Oakland Raiders|            null|           null

In [32]:
NFL_Guru_df.printSchema()

root
 |-- schedule_date: string (nullable = true)
 |-- schedule_season: integer (nullable = true)
 |-- schedule_week: string (nullable = true)
 |-- schedule_playoff: boolean (nullable = true)
 |-- team_home: string (nullable = true)
 |-- score_home: integer (nullable = true)
 |-- score_away: integer (nullable = true)
 |-- team_away: string (nullable = true)
 |-- team_favorite_id: string (nullable = true)
 |-- spread_favorite: double (nullable = true)
 |-- over_under_line: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- stadium_neutral: boolean (nullable = true)
 |-- weather_temperature: integer (nullable = true)
 |-- weather_wind_mph: integer (nullable = true)
 |-- weather_humidity: integer (nullable = true)
 |-- weather_detail: string (nullable = true)



In [33]:
url = "https://nflpredictionsdataviz.s3.amazonaws.com/stadiums.csv"
spark.sparkContext.addFile(url)
NFL_stadiums_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("stadiums.csv"), sep=",", header=True, inferSchema=True)
NFL_stadiums_df.show()

+------------------+--------------------+--------------------+--------+------------+
|         home_team|             stadium|    stadium_location|capacity|opening_year|
+------------------+--------------------+--------------------+--------+------------+
| Arizona Cardinals|  State Farm Stadium|   Glendale, Arizona|  63,400|        2006|
|              null|                null|                null|    null|        null|
|   Atlanta Falcons|Mercedes-Benz Sta...|    Atlanta, Georgia|  71,000|        2017|
|              null|                null|                null|    null|        null|
|  Baltimore Ravens|    M&T Bank Stadium| Baltimore, Maryland|  71,008|        1998|
|              null|                null|                null|    null|        null|
|     Buffalo Bills|       Bills Stadium|Orchard Park, New...|  71,608|        1973|
|              null|                null|                null|    null|        null|
| Carolina Panthers|Bank of America S...|Charlotte, North ...|  7

In [34]:
NFL_stadiums = NFL_stadiums_df.toPandas()
NFL_stadiums = NFL_stadiums.dropna()
NFL_stadiums.head(32)

,home_team,stadium,stadium_location,capacity,opening_year
0,Arizona Cardinals,State Farm Stadium,"Glendale, Arizona","63,400",2006.0
2,Atlanta Falcons,Mercedes-Benz Stadium,"Atlanta, Georgia","71,000",2017.0
4,Baltimore Ravens,M&T Bank Stadium,"Baltimore, Maryland","71,008",1998.0
6,Buffalo Bills,Bills Stadium,"Orchard Park, New York","71,608",1973.0
8,Carolina Panthers,Bank of America Stadium,"Charlotte, North Carolina","75,523",1996.0
10,Chicago Bears,Soldier Field,"Chicago, Illinois","61,500",1924.0
12,Cincinnati Bengals,Paul Brown Stadium,"Cincinnati, Ohio","65,515",2000.0
14,Cleveland Browns,First Energy Stadium,"Cleveland, Ohio","67,895",1999.0
16,Dallas Cowboys,At&T Stadium,"Arlington, Texas","80,000",2009.0
18,Denver Broncos,Empower Field at Mile High,"Denver, Colorado","76,125",2001.0


In [35]:
from pyspark.sql.functions import regexp_replace

In [36]:
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Washington Redskins', 'Washington Football Team'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Washington Redskins', 'Washington Football Team'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Washington Redskins', 'Washington Football Team'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Washington Redskins', 'Washington Football Team'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'St. Louis Rams', 'Los Angeles Rams'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'St. Louis Rams', 'Los Angeles Rams'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'St. Louis Rams', 'Los Angeles Rams'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'St. Louis Rams', 'Los Angeles Rams'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'St. Louis Cardinals', 'Arizona Cardinals'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'St. Louis Cardinals', 'Arizona Cardinals'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'St. Louis Cardinals', 'Arizona Cardinals'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'St. Louis Cardinals', 'Arizona Cardinals'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Phoenix Cardinals', 'Arizona Cardinals'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Phoenix Cardinals', 'Arizona Cardinals'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Phoenix Cardinals', 'Arizona Cardinals'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Phoenix Cardinals', 'Arizona Cardinals'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Oakland Raiders', 'Las Vegas Raiders'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Oakland Raiders', 'Las Vegas Raiders'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Oakland Raiders', 'Las Vegas Raiders'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Oakland Raiders', 'Las Vegas Raiders'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Los Angeles Raiders', 'Las Vegas Raiders'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Los Angeles Raiders', 'Las Vegas Raiders'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Los Angeles Raiders', 'Las Vegas Raiders'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Los Angeles Raiders', 'Las Vegas Raiders'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Houston Oilers', 'Tennessee Titans'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Houston Oilers', 'Tennessee Titans'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Houston Oilers', 'Tennessee Titans'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Houston Oilers', 'Tennessee Titans'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Tennessee Oilers', 'Tennessee Titans'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Tennessee Oilers', 'Tennessee Titans'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Tennessee Oilers', 'Tennessee Titans'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Tennessee Oilers', 'Tennessee Titans'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Tennessee Oilers', 'Tennessee Titans'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Tennessee Oilers', 'Tennessee Titans'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Tennessee Oilers', 'Tennessee Titans'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Tennessee Oilers', 'Tennessee Titans'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'San Diego Chargers', 'Los Angeles Chargers'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'San Diego Chargers', 'Los Angeles Chargers'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'San Diego Chargers', 'Los Angeles Chargers'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'San Diego Chargers', 'Los Angeles Chargers'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Baltimore Colts', 'Indianapolis Colts'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_home', regexp_replace('team_home', 'Baltimore Colts', 'Indianapolis Colts'))

NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Baltimore Colts', 'Indianapolis Colts'))
NFL_Guru_df = NFL_Guru_df.withColumn('team_away', regexp_replace('team_away', 'Baltimore Colts', 'Indianapolis Colts'))


In [37]:
NFL_guru_current = NFL_Guru_df.where(NFL_Guru_df.schedule_season>1978)
NFL_guru_current.show()

+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|           team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|     9/1/1979|           1979|            1|           false|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|              TB|           

In [38]:
from pyspark.sql import functions as F

NFL_guru_wins = NFL_guru_current.withColumn('home_win',
                                            F.when((F.col('score_home')>F.col('score_away')), 2)\
                                            .when((F.col('score_home')<F.col('score_away')), 1).otherwise(1)                                   
                                            )
NFL_guru_wins.show()

+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+--------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|           team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|home_win|
+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+--------+
|     9/1/1979|           1979|            1|           false|Tampa Bay Buccaneers|        31|        16|       Detroit Lions| 

In [39]:
cols = ('schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff', 'team_favorite_id', 'stadium_neutral','weather_humidity', 'over_under_line')
NFL_short = NFL_guru_wins.drop(*cols)
NFL_short.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|          null|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|          null|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|          null|       2|
|      Denver Br

In [50]:
NFL_fix_weather = NFL_short.withColumn('weather_detail', F.when(NFL_short.weather_detail=='DOME',1).otherwise(0))
NFL_fix_weather.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|             0|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|             0|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|             0|       2|
|      Denver Br

In [51]:
NFL_clean = NFL_fix_weather
NFL_clean.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|             0|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|             0|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|             0|       2|
|      Denver Br

In [52]:
NFL_clean = NFL_clean.dropna()
NFL_clean_df = NFL_clean.toPandas()
NFL_clean_df.head()

,team_home,score_home,score_away,team_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win
0,Tampa Bay Buccaneers,31,16,Detroit Lions,-3.0,Houlihan's Stadium,79,9,0,2
1,Buffalo Bills,7,9,Miami Dolphins,-5.0,Ralph Wilson Stadium,74,15,0,1
2,Chicago Bears,6,3,Green Bay Packers,-3.0,Soldier Field,78,11,0,2
3,Denver Broncos,10,0,Cincinnati Bengals,-3.0,Mile High Stadium,69,6,0,2
4,Kansas City Chiefs,14,0,Indianapolis Colts,-1.0,Arrowhead Stadium,76,8,0,2


In [53]:
NFL_clean_df.weather_detail.unique()

array([0, 1], dtype=int32)

In [54]:
results = []
home_teams = NFL_clean.select('team_home').distinct()
home_team_list = home_teams.rdd.map(lambda x: x[0]).collect()
visiting_teams = NFL_clean.select('team_away').distinct()
visiting_team_list = visiting_teams.rdd.map(lambda x: x[0]).collect()
teams_seen = []


In [55]:
home_team_list

['San Francisco 49ers',
 'Tennessee Titans',
 'Houston Texans',
 'Arizona Cardinals',
 'Jacksonville Jaguars',
 'Tampa Bay Buccaneers',
 'New England Patriots',
 'Buffalo Bills',
 'Kansas City Chiefs',
 'Philadelphia Eagles',
 'Indianapolis Colts',
 'New York Jets',
 'Cleveland Browns',
 'Chicago Bears',
 'Miami Dolphins',
 'Atlanta Falcons',
 'Green Bay Packers',
 'Carolina Panthers',
 'Denver Broncos',
 'Dallas Cowboys',
 'Los Angeles Chargers',
 'Las Vegas Raiders',
 'New Orleans Saints',
 'Detroit Lions',
 'Los Angeles Rams',
 'Seattle Seahawks',
 'Baltimore Ravens',
 'Pittsburgh Steelers',
 'Minnesota Vikings',
 'Cincinnati Bengals',
 'New York Giants',
 'Washington Football Team']

In [56]:
save = []
save = pd.DataFrame()
check_df = pd.DataFrame()
all_model_results = pd.DataFrame()

In [57]:
errornum = 1
successnum = 1 
for home_team in home_team_list:
  for visiting_team in visiting_team_list:
    if home_team == visiting_team:
      continue
    else:
      try:
        print("---------------------------")
        print (f"Success# {successnum}")
        successnum = successnum+1
        print (f"Teams: H: {home_team}, A: {visiting_team}")
        # Subset your dataframe
        temp_df = NFL_clean.filter(((NFL_clean.team_home == home_team) & (NFL_clean.team_away == visiting_team)) | ((NFL_clean.team_home == visiting_team) & (NFL_clean.team_away == home_team)))
          # Dump it to a pandas df
        temp_df_panda = temp_df.toPandas()
        temp_df_panda = temp_df_panda.drop(['team_home', 'team_away'], axis=1)
        check_df = temp_df_panda
        if len(temp_df_panda) > 10:     
          # Get dummies
          stadium_encoded = pd.get_dummies(temp_df_panda, columns=['stadium'])
          save = stadium_encoded
          stadium_encoded.head()
          X = stadium_encoded.drop(columns=['home_win'])
          y = stadium_encoded['home_win']

          # Split into X, Y and training/test sets

          X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1 )#,stratify = y

          # Fit model
          model = XGBClassifier()
          model.fit(X_train, y_train)


          # Predict
          y_pred = model.predict(X_test)
          accuracy = balanced_accuracy_score(y_test, y_pred)
          matrix = confusion_matrix(y_test, y_pred)

          results = {}
          # Put your results in a dataframe

          results['home_team'] = home_team
          results['away_team'] = visiting_team
          results['accuracy'] = accuracy
          print(results)
          # Store results
          #results.append()
          all_model_results = all_model_results.append(results, ignore_index=True) # = pd.concat([results])
        print("---------------------------")
      except Exception as e:
        print(f"Error#: {errornum}, Error Type: {e}")
        print(f"Teams: H: {home_team}, A: {visiting_team}")
        errornum = errornum +1
        

---------------------------
Success# 1
Teams: H: San Francisco 49ers, A: Tennessee Titans
{'home_team': 'San Francisco 49ers', 'away_team': 'Tennessee Titans', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 2
Teams: H: San Francisco 49ers, A: Houston Texans
---------------------------
---------------------------
Success# 3
Teams: H: San Francisco 49ers, A: Arizona Cardinals
{'home_team': 'San Francisco 49ers', 'away_team': 'Arizona Cardinals', 'accuracy': 0.8333333333333334}
---------------------------
---------------------------
Success# 4
Teams: H: San Francisco 49ers, A: Jacksonville Jaguars
---------------------------
---------------------------
Success# 5
Teams: H: San Francisco 49ers, A: Tampa Bay Buccaneers
{'home_team': 'San Francisco 49ers', 'away_team': 'Tampa Bay Buccaneers', 'accuracy': 0.9}
---------------------------
---------------------------
Success# 6
Teams: H: San Francisco 49ers, A: New England Patriots
{'home_team': 'San Francisco

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'home_team': 'Tennessee Titans', 'away_team': 'Atlanta Falcons', 'accuracy': 0.0}
---------------------------
---------------------------
Success# 47
Teams: H: Tennessee Titans, A: Green Bay Packers
---------------------------
---------------------------
Success# 48
Teams: H: Tennessee Titans, A: Carolina Panthers
---------------------------
---------------------------
Success# 49
Teams: H: Tennessee Titans, A: Denver Broncos
{'home_team': 'Tennessee Titans', 'away_team': 'Denver Broncos', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 50
Teams: H: Tennessee Titans, A: Dallas Cowboys
{'home_team': 'Tennessee Titans', 'away_team': 'Dallas Cowboys', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 51
Teams: H: Tennessee Titans, A: Las Vegas Raiders
{'home_team': 'Tennessee Titans', 'away_team': 'Las Vegas Raiders', 'accuracy': 0.625}
---------------------------
---------------------------
Success# 52
Teams: H: Tennessee

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'home_team': 'Tennessee Titans', 'away_team': 'New Orleans Saints', 'accuracy': 0.0}
---------------------------
---------------------------
Success# 54
Teams: H: Tennessee Titans, A: Detroit Lions
---------------------------
---------------------------
Success# 55
Teams: H: Tennessee Titans, A: Los Angeles Rams
{'home_team': 'Tennessee Titans', 'away_team': 'Los Angeles Rams', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 56
Teams: H: Tennessee Titans, A: Seattle Seahawks
{'home_team': 'Tennessee Titans', 'away_team': 'Seattle Seahawks', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 57
Teams: H: Tennessee Titans, A: Baltimore Ravens
{'home_team': 'Tennessee Titans', 'away_team': 'Baltimore Ravens', 'accuracy': 0.875}
---------------------------
---------------------------
Success# 58
Teams: H: Tennessee Titans, A: Pittsburgh Steelers
{'home_team': 'Tennessee Titans', 'away_team': 'Pittsburgh Steelers', 'accuracy'

...

{'home_team': 'Cincinnati Bengals', 'away_team': 'Miami Dolphins', 'accuracy': 0.75}
---------------------------
---------------------------
Success# 914
Teams: H: Cincinnati Bengals, A: Chicago Bears
---------------------------
---------------------------
Success# 915
Teams: H: Cincinnati Bengals, A: Atlanta Falcons
{'home_team': 'Cincinnati Bengals', 'away_team': 'Atlanta Falcons', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 916
Teams: H: Cincinnati Bengals, A: Green Bay Packers
---------------------------
---------------------------
Success# 917
Teams: H: Cincinnati Bengals, A: Carolina Panthers
---------------------------
---------------------------
Success# 918
Teams: H: Cincinnati Bengals, A: Denver Broncos
{'home_team': 'Cincinnati Bengals', 'away_team': 'Denver Broncos', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 919
Teams: H: Cincinnati Bengals, A: Dallas Cowboys
{'home_team': 'Cincinnati Bengals', 'a

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'home_team': 'New York Giants', 'away_team': 'Chicago Bears', 'accuracy': 0.0}
---------------------------
---------------------------
Success# 946
Teams: H: New York Giants, A: Atlanta Falcons
{'home_team': 'New York Giants', 'away_team': 'Atlanta Falcons', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 947
Teams: H: New York Giants, A: Green Bay Packers
{'home_team': 'New York Giants', 'away_team': 'Green Bay Packers', 'accuracy': 0.6666666666666666}
---------------------------
---------------------------
Success# 948
Teams: H: New York Giants, A: Carolina Panthers
---------------------------
---------------------------
Success# 949
Teams: H: New York Giants, A: Denver Broncos
---------------------------
---------------------------
Success# 950
Teams: H: New York Giants, A: Dallas Cowboys
{'home_team': 'New York Giants', 'away_team': 'Dallas Cowboys', 'accuracy': 0.9230769230769231}
---------------------------
---------------------------
Success# 9

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'home_team': 'Washington Football Team', 'away_team': 'Tennessee Titans', 'accuracy': 0.0}
---------------------------
---------------------------
Success# 964
Teams: H: Washington Football Team, A: Houston Texans
---------------------------
---------------------------
Success# 965
Teams: H: Washington Football Team, A: Arizona Cardinals
{'home_team': 'Washington Football Team', 'away_team': 'Arizona Cardinals', 'accuracy': 0.7424242424242424}
---------------------------
---------------------------
Success# 966
Teams: H: Washington Football Team, A: Jacksonville Jaguars
---------------------------
---------------------------
Success# 967
Teams: H: Washington Football Team, A: Tampa Bay Buccaneers
{'home_team': 'Washington Football Team', 'away_team': 'Tampa Bay Buccaneers', 'accuracy': 0.5}
---------------------------
---------------------------
Success# 968
Teams: H: Washington Football Team, A: New England Patriots
---------------------------
---------------------------
Success# 969

In [58]:
all_model_results

,home_team,away_team,accuracy
0,San Francisco 49ers,Tennessee Titans,0.500000
1,San Francisco 49ers,Arizona Cardinals,0.833333
2,San Francisco 49ers,Tampa Bay Buccaneers,0.900000
3,San Francisco 49ers,New England Patriots,0.750000
4,San Francisco 49ers,Kansas City Chiefs,1.000000
...,...,...,...
651,Washington Football Team,Detroit Lions,0.500000
652,Washington Football Team,Los Angeles Rams,0.500000
653,Washington Football Team,Seattle Seahawks,0.500000
654,Washington Football Team,Minnesota Vikings,0.416667


In [59]:
NFL_final_results = all_model_results.merge(NFL_stadiums, left_on='home_team', right_on='home_team')
NFL_final_results

,home_team,away_team,accuracy,stadium,stadium_location,capacity,opening_year
0,San Francisco 49ers,Tennessee Titans,0.500000,Levi's Stadium,"Santa Clara, California","68,500",2014.0
1,San Francisco 49ers,Arizona Cardinals,0.833333,Levi's Stadium,"Santa Clara, California","68,500",2014.0
2,San Francisco 49ers,Tampa Bay Buccaneers,0.900000,Levi's Stadium,"Santa Clara, California","68,500",2014.0
3,San Francisco 49ers,New England Patriots,0.750000,Levi's Stadium,"Santa Clara, California","68,500",2014.0
4,San Francisco 49ers,Kansas City Chiefs,1.000000,Levi's Stadium,"Santa Clara, California","68,500",2014.0
...,...,...,...,...,...,...,...
631,New York Giants,Detroit Lions,0.833333,MetLife Stadium,"East Rutherford, New Jersey","82,500",2010.0
632,New York Giants,Los Angeles Rams,0.583333,MetLife Stadium,"East Rutherford, New Jersey","82,500",2010.0
633,New York Giants,Seattle Seahawks,0.583333,MetLife Stadium,"East Rutherford, New Jersey","82,500",2010.0
634,New York Giants,Minnesota Vikings,1.000000,MetLife Stadium,"East Rutherford, New Jersey","82,500",2010.0


In [34]:
NFL_final_results.home_team.unique()

array(['San Francisco 49ers', 'Tennessee Titans', 'Houston Texans',
       'Arizona Cardinals', 'Jacksonville Jaguars',
       'Tampa Bay Buccaneers', 'New England Patriots', 'Buffalo Bills',
       'Kansas City Chiefs', 'Philadelphia Eagles', 'Indianapolis Colts',
       'New York Jets', 'Cleveland Browns', 'Chicago Bears',
       'Miami Dolphins', 'Atlanta Falcons', 'Green Bay Packers',
       'Carolina Panthers', 'Denver Broncos', 'Dallas Cowboys',
       'Los Angeles Chargers', 'Las Vegas Raiders', 'New Orleans Saints',
       'Detroit Lions', 'Los Angeles Rams', 'Seattle Seahawks',
       'Baltimore Ravens', 'Pittsburgh Steelers', 'Minnesota Vikings',
       'Cincinnati Bengals', 'New York Giants'], dtype=object)

In [1]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

NameError: ignored

In [ ]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9545454545454546

In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 9,  0],
       [ 1, 10]])

In [ ]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.90      1.00      0.91      0.95      0.95      0.92         9
          2       1.00      0.91      1.00      0.95      0.95      0.90        11

avg / total       0.96      0.95      0.96      0.95      0.95      0.91        20



In [ ]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({1: 29, 2: 30})

In [ ]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 9,  0],
       [ 0, 11]])

In [ ]:
print(classification_report_imbalanced(y_test, y_pred)) 

                   pre       rec       spe        f1       geo       iba       sup

          1       1.00      1.00      1.00      1.00      1.00      1.00         9
          2       1.00      1.00      1.00      1.00      1.00      1.00        11

avg / total       1.00      1.00      1.00      1.00      1.00      1.00        20



In [ ]:
ros = RandomOverSampler(random_state=1)
          X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
          model = LogisticRegression(solver='lbfgs', random_state=1)
          model.fit(X_resampled, y_resampled)